# IMPORTS

In [ ]:
%run ipynb_setup.ipynb

In [ ]:
%run class_Dataset.ipynb

In [ ]:
from typing import Tuple,Dict

import image_similarity_measures
from image_similarity_measures.quality_metrics import fsim,issm,psnr,rmse,sam,sre,ssim,uiq 
# https://github.com/up42/image-similarity-measures
# https://up42.com/blog/tech/image-similarity-measures

from joblib import Parallel, delayed

# CLASS DEF

In [ ]:
IMG_SIMILARITY_DICT   = Dict[str,float] # k,v dict with k = measure and v = similarity metric
IMG_SIMILARITY_DF     = pd.DataFrame    # cols of similarity metrics
IMG_SIMILARITY_SERIES = pd.Series       # column from IMG_SIMILARITY_DF
NOTEBOOK_PLOTS        = None

class ImageSearch():

    def __init__(
        self,
        dataset : Dataset,
        do_fsim : bool = False, # Feature-based similarity index (FSIM)                                   # 0 to 1, 0 == bad, 1 == identical                      # ascending=False
        do_issm : bool = False, # Information theoretic-based Statistic Similarity Measure (ISSM)
        do_psnr : bool = True,  # Peak signal-to-noise ratio (PSNR)                                                                                               # ascending=False
        do_rmse : bool = True,  # Root mean square error (RMSE)                                           # 0 to inf, 0 == identical, the smaller the better      # ascending=True
        do_sam  : bool = True,  # Spectral angle mapper (SAM)                                                                                                     # ascending=False
        do_sre  : bool = True,  # Signal to reconstruction error ratio (SRE)                                                                                      # ascending=False
        do_ssim : bool = True,  # Structural Similarity Index (SSIM)                                      # -1 to 1, -1 == bad, 1 == good, the larger the better  # ascending=False
        do_uiq  : bool = False, # Universal image quality index (UIQ)                                     # -1 to 1, -1 == bad, 1 == good, the larger the better  # ascending=False
        ) -> None :

        self.dataset  = dataset

        # default setting re computing which image similarity metrics
        self.do_fsim  = do_fsim # Feature-based similarity index (FSIM)
        self.do_issm  = do_issm # Information theoretic-based Statistic Similarity Measure (ISSM)
        self.do_psnr  = do_psnr # Peak signal-to-noise ratio (PSNR)
        self.do_rmse  = do_rmse # Root mean square error (RMSE)
        self.do_sam   = do_sam  # Spectral angle mapper (SAM)
        self.do_sre   = do_sre  # Signal to reconstruction error ratio (SRE)
        self.do_ssim  = do_ssim # Structural Similarity Index (SSIM)
        self.do_uiq   = do_uiq  # Universal image quality index (UIQ)
        
    # compute similarity metrics between some input image and some other image
    def img_similarity_pair(
        self,
        tgt_loc     : int,
        src_loc     : int            = None,  # one of src_loc or src_img need to be given
        src_img     : IMG            = None,  # one of src_loc or src_img need to be given
        plot_src    : bool           = True,  # should plot src img
        plot_tgt    : bool           = True,  # should plot tgt img
        grayscale   : bool           = False, # to avoid finding same item in different colour
        blur        : bool           = False, # to avoid finding same item with minor difference in listing
        blur_ksize  : Tuple[int,int] = (5,5), # the larger the stronger the smoothing
        ) -> IMG_SIMILARITY_DICT :
        '''
        d.img_similarity_pair(src_loc=3,tgt_loc=0)
        '''
        print(f'tgt_loc = {tgt_loc}')
        
        #####################################
        # srcimg
        #####################################
        # get srcimg
        if src_loc is not None:
            src_img = self.dataset.get_product_picture(
                loc        = src_loc,
                plot       = plot_src,
                grayscale  = grayscale,
                blur       = blur,
                blur_ksize = blur_ksize,
            )
        
        # srcimg scale
        scale_pct = 100 # percent of original img size
        src_img_width  = int(src_img.shape[1] * scale_pct / 100)
        src_img_height = int(src_img.shape[0] * scale_pct / 100)
        src_img_dim    = (src_img_width, src_img_height)

        #####################################
        # get other_img
        #####################################
        tgt_img = self.dataset.get_product_picture(
            loc        = tgt_loc,
            plot       = plot_tgt,
            grayscale  = grayscale,
            blur       = blur,
            blur_ksize = blur_ksize,
        )

        # resize to srcimg size
        resized_tgt_img = cv2.resize(tgt_img, src_img_dim, interpolation = cv2.INTER_AREA)

        #####################################
        # compute distances
        #####################################
        out = {}
        if self.do_fsim: out['fsim'] = fsim(src_img, resized_tgt_img)
        if self.do_issm: out['issm'] = issm(src_img, resized_tgt_img)
        if self.do_psnr: out['psnr'] = psnr(src_img, resized_tgt_img)
        if self.do_rmse: out['rmse'] = rmse(src_img, resized_tgt_img)
        if self.do_sam:  out['sam']  = sam(src_img,  resized_tgt_img)
        if self.do_sre:  out['sre']  = sre(src_img,  resized_tgt_img)
        if self.do_ssim: out['ssim'] = ssim(src_img, resized_tgt_img)
        if self.do_uiq:  out['uiq']  = uiq(src_img,  resized_tgt_img)

        #####################################
        # return dict
        #####################################
        return out
    
    # compute similarity of some input image to all other images
    def img_similarity_all(
        self,
        loc         : int            = None,  # 0 / index name to read
        plot_src    : bool           = True,  # should plot src img
        plot_tgt    : bool           = False, # should plot tgt img
        grayscale   : bool           = False, # to avoid finding same item in different colour
        blur        : bool           = False, # to avoid finding same item with minor difference in listing
        blur_ksize  : Tuple[int,int] = (5,5), # the larger the stronger the smoothing
        do_parallel : bool           = True,
        ) -> IMG_SIMILARITY_DF :
        
        #####################################
        # src_img
        #####################################
        # get src_img
        src_img = self.dataset.get_product_picture(
            loc        = loc,
            plot       = plot_src,
            grayscale  = grayscale,
            blur       = blur,
            blur_ksize = blur_ksize,
        )
                
        #####################################
        # function to compute distances
        #####################################
        def JOBLIB_PARALLEL_FUNC(
            tgt_loc : int,
            ) -> IMG_SIMILARITY_DICT :
            return self.img_similarity_pair(
                tgt_loc    = tgt_loc,
                src_loc    = None,
                src_img    = src_img,
                plot_src   = plot_src,
                plot_tgt   = plot_tgt,
                grayscale  = grayscale,
                blur       = blur,
                blur_ksize = blur_ksize,
            )
        
        # parallelize the compute
        if do_parallel:
            out = Parallel(n_jobs=-1)(delayed(JOBLIB_PARALLEL_FUNC)(tgt_loc) for tgt_loc in self.dataset.df.index)
        else:
            out = [JOBLIB_PARALLEL_FUNC(tgt_loc) for tgt_loc in self.df.dataset.index]
        
        #####################################
        # return df
        #####################################
        return pd.DataFrame(out,index=self.dataset.df.index)

    # show distribution of image similarity metrics for our dataset
    def img_similarity_plot(
        self,
        img_similarity_df : IMG_SIMILARITY_DF,
        ) -> NOTEBOOK_PLOTS :
        df = img_similarity_df.copy()
        df = df.replace([np.inf, -np.inf],0)
        df.hist(figsize=(20,15))

In [ ]:
'''
im_search=ImageSearch(dataset=Dataset())
im_search.img_similarity_pair(tgt_loc=2,src_loc=3)
'''
None